# 2. házi feladat

A második házi feladat során egy egyszerű Multi-layer Perceptron (MLP) típusú neuronhálót kell betanítani és kiértékelni egy regressziós és egy klasszifikációs feladaton a Keras könyvtár segítségével.

A házi feladat több részfeladatból áll (A - I), melyek egymásra épülnek. Az A, B, C, D, F és H részfeladatokból több változat van, ezekből mindenkinek egyet-egyet kell megoldania. **Arról, hogy ki melyiket oldja meg, az alábbi linken található lista ad tájékoztatást** Neptun-kód - feladatok listája formában:

http://nipg28.inf.elte.hu/~vavsaai@nipg.lab/annbsc22_p1_hw2/annbsc22_p1_hw2_assignments.txt

A feladat automatikus tesztelőszkripttel rendelkezik. Éppen ezért, **ne változtasd meg a notebook struktúráját**, ne szedd több részre, illetve ne módosítsd a megírt részeket, csak a megoldásaidat írd be a megfelelő helyre, különben nem fogjuk tudni értékelni! A megoldások az `# Implement your solution BELOW` és `# Implement your solution ABOVE` sorok közé írandók. Ha a megoldott részfeladatok futtatásakor a "Tester: .... OK" felirat megjelenik, _vélhetően_ jó a megoldás. Azonban, a tesztelőszkript nem képes a megoldás helyességét minden aspektus szerint ellenőrizni. Így elképzelhető, hogy beadás után további javításokat kérünk a megoldásban. A beadott házi feladatok plágium ellenőrzésen esnek át. Másolás gyanújának felmerülése esetén mind a másolat(ok), mind az azt/azokat ihlető eredeti megoldások visszautasíthatók.

**!!! A feladatok megoldását, vagy az arra mutató linkeket bárhol közzétenni tilos !!!**

Elakadás esetén javasolt átnézni a Canvason megosztott segédanyag notebookokat és az előadások anyagát. Továbbá, a Canvas fórumon kérhető segítség, de másik hallgató által legfeljebb olyan jellegű válasz adható, hogy melyik függvényt érdemes megpróbálni, vagy például egy hasonló példához hogyan kellene a tömböt indexelni. **Akár kérdésben, akár válaszban a megoldás jelentős részét tartalmazó kódrészletet közzétenni tilos.**

**Megoldás és beküldés menete:**

A közzétett, feladatokat tartalmazó Google Colab notebook csak "view" jogosultságokkal van megosztva, így, bár a notebook szerkeszthetőnek látszik, az úgynevezett "Playground" módban nyílik meg, ahol a változtatások nem lesznek mentve. Mindenkinek le kell másolnia a saját Drive-jára a notebook-ot (Google Colab file menü -> Save a copy to Drive): a másolat már maradandóan szerkeszthető és futtatható lesz. **A kész megoldást tartalmazó notebook-ot le kell tölteni**  (Google Colab file menü -> Download .ipynb), **majd a Canvas-ban, a kiírt feladatnál feltölteni.** Link nem adható be, csak az .ipynb fájl. Alternatívaként, Google Colab helyett saját gépen futó Python-nal és Jupyter-rel is szerkeszthetők és futtathatók .ipynb notebookok.

**Első határidő:** 2021. április 29., péntek, 23:59.

**Végső határidő:** 2021. május 10., kedd, 23:59. - **!!! A végső határidőig mindenkinek az oktatók által elfogadható, beadott megoldással kell rendelkeznie, ezt követően pótlásra és/vagy javításra nincs lehetőség. !!!**

**Pontozás:** A házi feladat helyes megoldása feltétele a jegyszerzésnek. Az, aki az első határidőig teljes, legfeljebb apró hibákat tartalmazó megoldást adott be, 2 extra pontot kap.

**Kikötés:** Ahol csak lehet, hatékony, vektorizált megoldásokat várunk.



Tesztelő szkript letöltése, importok.

In [ ]:
# Download tester
!rm annbsc22_p1_hw2_tester.py
!wget http://nipg28.inf.elte.hu/~vavsaai@nipg.lab/annbsc22_p1_hw2/annbsc22_p1_hw2_tester.py

import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.layers
import tensorflow.keras.activations
import tensorflow.keras.callbacks

import matplotlib.pyplot as plt

# import tester after importing tensorflow, to make sure correct tf version is imported
from annbsc22_p1_hw2_tester import Tester


###**A**: Az adatbázis letöltése

**A/1 - A/6**: A tesztelőszkript a számodra szükséges adatbázist tölti le és a részedre kiosztott feladatok helyességét fogja tesztelni. Ehhez az alábbi kódblokk tetején, a Tester példányosításakor paraméterként a Neptun-kódodat kell megadni egy sztringként, pl:
`tester = Tester('DK7UAQ')`

**Információ az adatbázisokról**:
Az adatbázisok a UCI Machine Learning Repository-ról származnak.
- **A/1**: Különböző vegyületekre (a vegyületek tulajdonságai alapján) próbáljuk megbecsülni azt a mennyiséget, ami az esetek 50%-ában megöli a _Pimephales promelas_ fajtájú halat. http://archive.ics.uci.edu/ml/datasets/QSAR+fish+toxicity
- **A/2**: Két portugál középiskolában a tanulók évvégi matematika jegyét (0-20) próbáljuk megbecsülni a tanulók szociális és pénzügyi háttere alapján. http://archive.ics.uci.edu/ml/datasets/Student+Performance
- **A/3**: Lásd A/2, matematika helyett portugál nyelv tárgyból.
- **A/4**: Lásd A/2. Az A/2. és A/3.-ban használt adatbázisok esetében az input változók közt megtalálhatók a tanulók első- és második harmadévvégi osztályzata az adott tárgyból. Ezekkel nyilvánvalóan erősen korrelál az év végi jegy, így nehezíti a feladatot, ha ezeket az input változókat elhagyjuk. Az A/4 és A/5. adatbázisaiból ezek a változók hiányoznak, így jelentősen gyengébb eredmény várható.
- **A/5**: Lásd A/4, matematika helyett portugál nyelv tárgyból.
- **A/6**: Betontömbök nyomószilárdságát (~1-100 MPa) próbáljuk megbecsülni a betontömbök összetevőinek mennyiségéből, előállítási körülményeikből és korukból. http://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

A tesztelőszkript letölti a szöveges formában tárolt adatbázist és elhelyezi a `content` sztringben. Alább, kiírjuk a hosszát és az első 500 karaktert is a sztringből. Az adatbázis fájlok egy-egy sora adja meg az egy-egy mintaelemhez tartozó adatokat, ahol az utolsó érték a címke, míg a többi érték az input változók értékeit kódolja. Az adatbázis első sora a változók neveit tartalmazza.

A feladatod, hogy alakítsd át a `content` sztringet az adatbázis input változóit és címkéit tartalmazó tömbbé. A tömbök típusa np.float32 (lebegőpontos) típusú legyen! Az input változók értékeit helyezd el a `features` (n_mintaelem, n_valtozo) alakú tömbben, míg a címkéket a `labels` (n_mintaelem,) alakú tömbben!



In [ ]:
tester = Tester('YOUR_NEPTUN_CODE')
content = tester.get_dataset_content()

print(len(content))
print(content[:500])


In [ ]:
# implement your solution BELOW





# implement your solution ABOVE
tester.test('dataset_shape', features, labels)


## **B**: Tanító-, validációs- és teszthalmaz szétválasztása

Keverd meg véletlenszerűen a mintaelemeket a `features` és `labels` tömbökben (természetesen a két tömb permutálása legyen azonos)! Ez azért szükséges, mert az adatbázisban az elemek lehet, hogy rendezettek pl. a címkék szerint. Megkeverés nélkül, a szétválasztott halmazokban jelentősen különbözne a címkék eloszlása.

Ezt követően válaszd szét a tömböket tanító-, validációs- és teszthalmazra!

A szétválasztás történjen az alábbi arányokkal:
- **B/1**.: 70% / 15% / 15%
- **B/2**: 60% / 20% / 20%
- **B/3**: 50% / 25% / 25%

A halmazokat tartalmazó három-három input- és címketömb változókat nevezd el `x_unnorm_train`, `x_unnorm_val`, `x_unnorm_test`, `y_train`, `y_val`, `y_test`-nek sorrendben.

In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('dataset_split', x_unnorm_train, x_unnorm_val, x_unnorm_test,\
                             y_train, y_val, y_test)


## **C**: Az input változók azonos nagyságrendre hozása

A változók azonos nagyságrendre hozása az alábbi technikával történjen:
- **C/1**: sztenderdizáció (0 átlag, 1 szórás)
- **C/2**: min-max skálázás (0 minimum, 1 maximum)

A változókat egymástól függetlenül kell skálázni. A skálázáshoz szükséges átlagot és szórást / minimumot és maximumot a tanítóhalmazon számold ki, majd alkalmazd őket a validációs és a teszt halmazra. Ez azért szükséges, mert a teszthalmazról azt feltételezzük, hogy ismeretlen a betanítás során, pontosan azért, hogy semmi esetre se tudjuk hozzáigazítani a tanított modellünket.

**A feladatot *NumPy* segítségével kell megoldani. Ennek a részfeladatnak a megoldásához nem megengedett a *Keras* és a *TensorFlow* könyvtárnak és azok almoduljainak használata.**

A skálázott halmazokat tartalmazó három változót nevezd el `x_train`, `x_val`, `x_test`-nek sorrendben!

In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('dataset_rescale', x_train, x_val, x_test)


## **D**: A regressziós neuronháló definiálása Keras-ban

Használd a `keras.models.Sequential()` osztályt a háló definiálásához! A háló architektúráját az alábbiak alapján alakítsd ki:
- **D/1**: Kétrétegű háló, az első réteg neuronjainak száma 50, aktivációs függvénye ReLU. Az első réteg után alkalmazz dropout-ot neurononként 30%-os valószínűséggel!
- **D/2**: Négyrétegű háló, a neuronok száma rendre 20, 20, 10, aktivációs függvényük ReLU. Az egyes rétegek közé tégy dropout rétegeket, melyek 20%-os valószínűséggel nulláznak ki egy-egy elemet!
- **D/3**: Háromrétegű háló, a neuronok száma rendre 50, 30. Aktivációs függvényként használj sorban tanh-t, majd ReLU-t!

Minden esetben csak teljesen összekötött (dense) rétegeket, valamint dropout rétegeket kell használnod. Az aktivációs függvényeket a teljesen összekötött rétegek `activation` paramétereként add meg (azaz most ne használj külön Acitvation rétegeket). A rétegek számába beleértendő a kimenetre képző, utolsó réteg is, melynek méretét a címke mérete határozza meg. Alkalmazd a tanult aktivációs függvényt az utolsó rétegen, függően a feladat típusától.
A modellt fordítsd le a `compile()` függvény segítségével: a költségfüggvény legyen a szokásos, regresszióhoz használatos átlagos négyzetes eltérés (MSE), optimizer algoritmusként pedig alkalmazd az SGD algoritmust megfelelő tanulási rátával!
A regressziós neuronhálót tartalmazó keras.models.Sequential() típusú változót nevezd el `reg_model`-nek!

In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('reg_model_architecture', reg_model)

## **E**: A háló betanítása regressziós feladatra

Tanítsd be a neuronhálót a tanítóhalmazon! Használj early stopping-ot a validációs halmazzal! A tanulási rátát, epoch-ok maximális számát, a batch méretet, az early stopping `patience` paraméterét szabadon átállíthatod. Próbálgathatsz különböző kombinációkat, hogy jobb eredményt érj el.

- Rajzold ki, hogyan alakult a tanulási és validációs költség a betanítás során! Ehhez használhatod a `matplotlib` könyvtárat, példát találsz a 8. előadás notebookjában.
- A négyzetes (MSE) költség mellett az átlagos abszolút hiba (Mean Absolute Error, MAE) tanító- és validációs halmazon történő alakulását is rajzold ki egy másik grafikonon! A grafikonok y tengelyének megjelenített értékhatárait úgy állítsd be, hogy mindegyik idősor látható legyen és könnyen ki lehessen venni a költségek alakulását a tanítás vége felé is!
- Számold ki a betanított modell négyzetes hibáját és az átlagos abszolút hibát a teszthalamzon, majd add értékül ezeket az értékeket a `test_mse` és `test_mae` változóknak!
- Válassz néhány példát a teszthalmazból és becsülj hozzájuk címkét a betanított modellel, majd írasd ki a becsült és a megfelelő helyes címkéket! Így példákon is láthatjuk, hogy mennyire ad jó becsléseket a neuronhálónk.



In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('reg_model_learning', test_mse, test_mae)

## **F**: A regressziós adatbázis átalakítása klasszifikációs adatbázissá

Alakítsd át a regressziós adatbázist, hogy alkalmas legyen
klasszifikációs feladathoz! A kategóriákat az alábbiak alapján alakítsd ki :

*   **F/1**: 3 kategória. A kategóriákat úgy alakítsd ki, hogy a regressziós adatbázis címkéinek intervallumán felveszel két küszöbértéket. Az első kategóriába a kisebb küszöbérték alatti, a másodikba a két küszöbérték közti és a harmadikba a nagyobb küszöbérték feletti címkével rendelkező mintaelemek kerüljenek.
Tehát, például ha a regressziós adatbázis címkéi 3 és 8 közt vannak, akkor a két küszöbértéknek választhatjuk az 5.2-t és a 6.1-et. Ekkor az 1. kategóriába az 5.2-nél kisebb címkéjű elemek, a 2. kategóriába az 5.2 és 6.1 közti címkéjű elemek és a 3. kategóriába a 6.1-nél nagyobb címkéjű elemek kerülnek. Azonos címkéjű elemek mindenképpen azonos kategóriába kell, hogy kerüljenek.

*   **F/2**: 4 kategória. A kategóriákat úgy alakítsd ki, hogy a regressziós adatbázis címkéinek intervallumán felveszel három küszöbértéket. Az első kategóriába a kisebb küszöbérték alatti, a másodikba az első és második küszöbérték közti, a harmadikba a második és a haramdik küszöbérték közti, és a negyedikbe a harmadik küszöbérték feletti címkével rendelkező mintaelemek kerüljenek.
Tehát, például ha a regressziós adatbázis címkéi 3 és 8 közt vannak, akkor a három küszöbértéknek választhatjuk a 3.8-at, a 6.1-et és a 7.5-öt. Ekkor az 1. kategóriába az 3.8-nál kisebb címkéjű elemek, a 2. kategóriába az 3.8 és 6.1 közti címkéjű elemek, a 3. kategóriába az 6.1 és 7.5 közti címkéjű elemek és a 4. kategóriába a 7.5-nél nagyobb címkéjű elemek kerülnek. Azonos címkéjű elemek mindenképpen azonos kategóriába kell, hogy kerüljenek.


Válaszd ki a küszöbértékeket úgy, hogy a kategóriábákba **megközelítőleg** ugyanannyi elem kerüljön! Hozd létre az új kategóriacímkéket az `y_train`, `y_val`, `y_test` tömbökből. Kerüljenek ezek a `y_cat_train`, `y_cat_val`, `y_cat_test` változókba!



In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('cl_dataset', y_cat_train, y_cat_val, y_cat_test)

## **G**: Kategóriacímkék átalakítása one-hot kódolásra

Kettőnél több kategóriás (multi-class) klasszifikációnál minden kategóriához egy valószínűséget becslünk, azaz a háló kimenete akkora méretű, ahány kategóriánk van. Így tehát az igazi címkéink ugyanekkora méretű one-hot kódolású vektorok lesznek, ezeket címkevektorokként használva fogjuk betanítani a hálót.

Készítsd el a one-hot kódolású címkevektor tömböket a kategóriacímkéket tartalmazó `y_cat_train`, `y_cat_val`, `y_cat_test` tömbökből! Az új tömbök kerüljenek a `y_onehot_train`, `y_onehot_val`, `y_onehot_test` változókba! A tömbök alakja így (n_samples, n_categories), típusuk legyen `np.float32`!


In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('cl_onehot', y_onehot_train, y_onehot_val, y_onehot_test)

## **H**: A klasszifikációs neuronháló definiálása Keras-ban

Használd a `keras.models.Sequential()` osztályt a háló definiálásához! A háló architektúráját az alábbiak alapján alakítsd ki:
- **H/1**: Kétrétegű háló, az első réteg neuronjainak száma 50, aktivációs függvénye ReLU. Az első réteg után alkalmazz dropout-ot neurononként 30%-os valószínűséggel!
- **H/2**: Négyrétegű háló, a neuronok száma rendre 20, 20, 10, aktivációs függvényük ReLU. Az egyes rétegek közé tégy dropout rétegeket, melyek 20%-os valószínűséggel nulláznak ki egy-egy elemet!
- **H/3**: Háromrétegű háló, a neuronok száma rendre 50, 30. Aktivációs függvényként használj sorban tanh-t, majd ReLU-t!

Minden esetben csak teljesen összekötött (dense) rétegeket, valamint dropout rétegeket kell használnod. Az aktivációs függvényeket a telejesen összekötött rétegek `activation` paramétereként add meg (azaz most ne használj külön Acitvation rétegeket). A rétegek számába beleértendő a kimenetre képző, utolsó réteg is, melynek méretét a címke mérete határozza meg. Alkalmazd a tanult aktivációs függvényt az utolsó rétegen, függően a feladat típusától!
A modellt fordítsd le a `compile()` függvény segítségével: a költségfüggvény legyen a szokásos, multi-class klasszifikációhoz használatos (categorical) crossentropy, optimizer algoritmusként pedig alkalmazd az SGD algoritmust megfelelő tanulási rátával!
A klasszifikációs neuronhálót tartalmazó keras.models.Sequential() típusú változót nevezd el `cl_model`-nek.

In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('cl_model_architecture', cl_model)

## **I**: A háló betanítása (multi-class) klasszifikációs feladatra

Tanítsd be a neuronhálót a tanítóhalmazon! Használj early stopping-ot a validációs halmazzal! A tanulási rátát, epoch-ok maximális számát, a batch méretet, az early stopping `patience` paraméterét szabadon átállíthatod. Próbálgathatsz különböző kombinációkat, hogy jobb eredményt érj el.

- Rajzold ki, hogyan alakult a tanulási és validációs költség a betanítás során! Ehhez használhatod a `matplotlib` könyvtárat, példát találsz a 8. előadás notebookjában.
- A crossentropy költség mellett, a helyesen klasszifikált mintaelemek arányának (_accuracy_) tanító- és validációs halmazon történő alakulását is rajzold ki egy másik grafikonon! A grafikonok y tengelyének megjelenített értékhatárait úgy állítsd be, hogy mindegyik idősor látható legyen és könnyen ki lehessen venni a költségek alakulását a tanítás vége felé is!
- Számold ki a betanított modell crossentropy költségét és az accuracy metrikát a teszthalamzon, majd add értékül ezeket az értékeket a `test_ce` és `test_acc` változóknak!
- Számold ki a multiclass _Fals pozitív arány_ (FPR) és _Fals negatív arány_ (FNR) metrikákat a teszthalmazon a betanított modelledre, majd add értékül ezeket az értékeket a `test_fpr` és `test_fnr` változóknak! Az FPR és FNR kiszámításához az első házi feladat B/1. és B/2. feladataiban található képletek és leírás segíthetnek. A **saját**, 1. házi feladatban beadott FPR vagy FNR metrikát kiszámító kódodat természetesen felhasználhatod itt is.

In [ ]:
# implement your solution BELOW





# implement your solution ABOVE

tester.test('cl_model_learning', test_ce, test_acc, test_fpr, test_fnr)
tester.print_all_tests_successful()